# Pipeline RNA-seq

## CUTADAPT

In [ ]:
cutadapt -j 12 -q 30,30 -m 35 -O 7 \
        -b XXXXXXXX -B XXXXXXXXXXXX

# Approche K-mer

## DEKUPL

#### EXPRESSION DIFFERENTIEL

#### INPUT

input:
- config.json
- dekupl-run.img

L'outil DEKUPL que nous avons utilisé est un pipeline sous le format snakemake qui va prendre en entrée un fichier de configuration, contenant les informations utilisé par l'ensemble des outils comme jellyfish (pour la table d'abondance), les seuils de pvalue et de FDR (lors de l'analyse statistique avec le packages DEseq2) etc (tout est expliqué dans la documentation du github de DEKUPL => https://github.com/Transipedia/dekupl-run

Exemple des paramètre du fichier config.json:

diff_methods => choix de la méthode pour l'analyse différentiel

data_type => Permet de lancé l'outil DEKUPL sans utilisé de référence (L'outils utilise la référence humaine (possibilité de désactivé).

### COMMANDE

Commande Bash pour lancé l'outils DEKUPL:

In [ ]:
%%bash
singularity run image/dekupl-run.img --configfile config.json -j 9 --ressources ram=20000 -p

### OUTPUT

L'outils créer plusieurs fichier intermédaire comme un fichier contenant l'ensemble des pvalues associé à chaque kmer, les tables de comptages présentes pour chaque échantillons, une matrice des abondances de kmer, une matrice contnant les kmer trouvé De en fonction des seuil choisis en input, avec les log2FC, les pvalue associé.

OUTPUT: diff-counts.tsv  

|tag|pvalue|meanA|meanB|log2FC|RC1|RC2|RC3|RC5|RC6|RC7|RC8|RC9|RC10|RS1|RS2|RS3|RS4|RS5|RS6|RS7|RS8|RS10|
|---|------|-----|-----|------|---|---|---|---|---|---|---|---|----|---|---|---|---|---|---|---|---|----|
|AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC|4.20378804183321e-67|8.63539545864017|69.663581259201|3.02693059366878|6.21217947483477|4.88860953977233|8.72149434084036|9.16014388087816|8.69132810322318|12.8018115985836|13.9431607053845|5.89953533784795|7.40029614639669|74.9162700511193|72.4851902807309|71.7218111765533|68.6567983961772|60.9638382832583|72.5478816018572|63.5218693401627|68.9376846416892|73.2208875612604|
|AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGT|0.000219182949844764|0.498349895691011|5.337454146431|3.55810867156966|0|1.39674558279209|0|0|0|0|1.74289508817306|0|1.34550839025394|7.34473235795288|11.3702259263892|7.17218111765533|0|6.35039982117274|4.9464464728539|4.6479416590363|0	6.20515996281868|



### ANNOTATION FILE

Dans cette partie nous utilisons quelques commande bash dans le but de créer notre fichier d'annotations final pour la suite de l'analyse.

In [ ]:
%%bash
gene.gff = 'africanRice_nrctgs_V1.blastp.ipr.gff'
gene2.gff = 'oryza_glaberrima_core_3_87_4.chr8.chr.renamed.blastp2.iprsca.gff' 

L'ensemble des fichier d'annotation est fournis par Christine Tranchant dans le cadre de son sujet de thèse.
Nous avons donc deux fichier d'annotation qui ont environ 22 765 gènes Pour le fichier {gene.gff} et environ 40 553 gène annoter pour le fichier {gene2.gff}

Nous avons décider d'utiliser les 2 fichier d'annotation, ainsi on concatène les deux fichier ensemble pour créer le ficheir gene_60k.gff, qui contient donc nos 60 000 gène annoter.

In [ ]:
%%bash
cat ${gene.gff} ${gene2.gff} > file_inter.gff

In [ ]:
%%bash
awk '$3 == "gene"' file_inter.gff > gene_60k.gff

## ALIGNEMENT DES K-MERS SUR GENOME

Nous avons donc notre matrice de kmer différentiellement exprimée, on souhaite maintenant procéder à une étape de mapping. Pour se faire nous allons créer un fichier fastq artificielle à partir de nos kmers.

Pour se faire nous récupérons chaque k-mer, afin de créer un fichier fasta contenant en identifiant un numéro_k-mers et en séquence le k-mer.

### Création kmer.fastq

Ce script prend un fichier contenant les tags 'k-mer' et va créer un fichier fasta à partir des k-mers contenus dans le fichier d'entrée.

Paramètre:  
-i INPUT FILE  
-o OUTPUT FILE  

le fichier saisie avec le paramètre -i doit être de type tsv avec au minimum comme première colonne les tags 'kmers'. Le ficheir d'entrée peut ne pas contenir d'header, ou sinon l'header pour la colonnes des tags doit être contig ou tag

La sortie DE-KUPL l'outils que nous avons utilisé pour les analyses précedetes fournis un format acceptable pour notre script notament pour les fichier diff-counts et raw_pvalues.tsv


EXEMPLES INPUT:

EX 1:  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGT  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGG  


EX2:  
tag  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGT  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGG  

EX3:  
contig  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGT  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGG  

In [ ]:
%%bash
input='diff-counts.tsv'
output='kmer_DE.fasta'

In [1]:
%%bash
python fasta_to_kmer.py -i ${input} -o ${output}

NameError: name 'file_in' is not defined

Extrait du fichier fatsq possible:

>1_tags_AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC  
>2_tags_AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAG  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAG  
>3_tags_AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAT  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAT  
>4_tags_AAAAAAAAAAAAAAAAAAAAAAAAAAAAACA  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAACA  
>5_tags_AAAAAAAAAAAAAAAAAAAAAAAAAAAAACC  
AAAAAAAAAAAAAAAAAAAAAAAAAAAAACC  

Ensuite nous procédons à l'alignement des k-mers à partir du fichier fasta créé.

## BWA aln

Nous avons choisis d'utiliser bwa aln en fonction des performances de nos jeu de données, nous l'avons comparer avec bwa mem2. Etant donné qu'on travail avec des court reads (100 nt max). BWA aln est le plus adapté.

Les inputs doivent contenir les chemins absolues des répertoires ou sont contenues la réferences et le fichier avec la séquence fasta des kmers.
Les inputs peuvent être compréssé au format gz.

Voici un exemple du format des entrées du script bwa.sh:

ref => nas3:/data3/projects/integration/ref/REFPANG-CT.fasta

fasta => nas3:/data3/projects/integration/rnaseq_riz/output/clean/testing/mapping_DE/kmer_DE.fasta.gz

ATTENTION: Si vous êtes sur le cluster Itrop, il est impératif d'indiquer le nas dans lequel les données d'entrées se trouvent.


In [ ]:
%%bash
sbash bwa.sh nas3:/data3/projects/integration/OglaRS2.ADWL02-allCtgsIRIGIN_TOG5681.dedup8095-NR.fasta nas3:/data3/projects/integration/rnaseq_riz/output/clean/testing/mapping_DE/kmer_DE.fasta.gz

Voici le détail de ce que fait le code bwa.sh vous trouvrez les paramètres, ainsi que les outils utilisé.

### Association des k-mers aux gènes

Cette étape à pour but de d'asocier les k-mers aux gènes contenu dans le fichier d'annotation du génome, en créant un fichier de sortis au format bed.

Pour l'étape d'intersect entre le fichier bam et le fichier gff, on utilise l'outils bedtools intersect.

Pour l'etape d'intersect on charge le module suivant:

In [ ]:
%%bash
module load bioinfo/bedtools/2.30.0

In [ ]:
%%bash

#DECLARATION DES VARIABLES
#
#à modifier selon vos données
#
bam="REFPANG-CT_VS_kmer_DE.bam"
genes_gff="gene_60k.gff"
output="REFPANG-CT_VS_kmer_DE_intersect_gene60k.bed"

#ETAPE: d'intersection entre le fichier d'annotation (gff) et le fichier bam qu'on vient de génerer
# -abam fichier bam contenant le mapping des kmers
# -b fichier d'annotation (gff)
# -bed => sortie en format bed
# -header garde les en-tête des fichier a et b d'entrée
# -wb garde les informations du fichier d'entré b
# -waq garde les informations du fichier d'entrée a

bedtools intersect -abam ${bam} -b ${genes_gff} -bed -header -wb -wa > ${output}


## TRAITEMENT POST-DEKUPL

### Gene Analysis

Dans cette partie nous allons traiter l'information au niveau du gène, comme par exemple le nombre de k-mers par gènes ou encore quel gène ont été associés à des k-mers.

#### bedAnalysis

Dans ce scirpt nous allons utilisé le fichier bed qui est créer suite à l'intersection entre le fichier d'annotation et notre fichier bam issus du mapping.

Le fichier input doit être de la forme :  

|Region kmer|Start kmer|Stop kmer|Tag|MapQ|Strand kmer|Start kmer|Stop kmer|?|?|CIGAR|?|Region gff|Type|Start Gene|Stop Gene|?|Strand Gene|?|ID|
|-----------|----------|---------|---|----|-----------|----------|---------|-|-|-----|-|----------|----|----------|--------|-|-----------|-|--|
|Chr01|7120|7151|32957537_tags_CGACGACTTCCCCACTAGGAAACACGACGGA|37|+|7120|7151|0,0,0|1|31,|0,|Chr01ensembl|gene|7074|14927|.|+|.|ID=Oglab_000002;Name=Oglab_000002;biotype=protein_coding;Note=Similar to RABGAP22: Rab GTPase-activating protein 22 (Arabidopsis thaliana OX%3D3702);Dbxref=InterPro:IPR000195,Pfam:PF00566;|
|Chr01|7124|7155|14920424_tags_CGCCTCCGTCGTGTTTCCTAGTGGGGAAGTC|37|-|7124|7155|0,0,0|1|31,|0,|Chr01ensembl|gene|7074|14927|.|+|.|ID=Oglab_000002;Name=Oglab_000002;biotype=protein_coding;Note=Similar to RABGAP22: Rab GTPase-activating protein 22 (Arabidopsis thaliana OX%3D3702);Dbxref=InterPro:IPR000195,Pfam:PF00566;|

C'est un format de sortie fournie par bedtools intersect, en utilisant les paramètres décris dans la partie "Association des k-mers aux gènes". 

Le script bedAnalysis prend comme entrée un seul fichier au format bed. (exemple de la structure dans la cellules précedente)

-b bed format file
-s entry if in last colums have note (function, similar gene etc)
-mapq choose your values mapq filtrer
-o name of output tsv
-k numbers of kmer reccurence for concidere a gene

In [ ]:
%%bash
kmer.bed = 'kmer_DE.bed'

In [ ]:
%%bash
python bed_Analysis.py -b kmer.bed -s -k 10 -o liste_kmer_gene_DE

#### Output

Le script fournis une table contenant chaque région avec le nombre de gènes associés.

|REGIONS|nb_gene|
|-------|-------|
|AI_67745|1|
|AK_86862|1|
|CL_97942|1|
|CP_155140|1|
|Chr01|2448|
|Chr02|2080|
|Chr03|2344|
|Chr04|1522|
|Chr05|1442|


Et une seconde table contenant les k-mers associés à chaque régions.

|REGIONS|nb_kmer|
|-------|-------|
|AI_67745|18|
|AK_86862|10|
|AM_61996|7|
|BE_99833|1|
|CB_109929|1|
|CL_97942|20|
|CP_155140|108|
|CS_116200|4|
|Chr01|504228|


Ainsi qu'un fichier TSV récapitulant l'ensemble des informations liés aux gènes.
Comme la région, la taille du gène ou encore la fonctions et la famille traduite à partir du gène.

|Gene|region|START|STOP|length|nb_kmers|Similar|Note|
|----|------|-----|----|------|--------|-------|----|
|Oglab_000002|Chr01|7074|14927|7853|162|RABGAP22|Similar to RABGAP22: Rab GTPase-activating protein 22 (Arabidopsis thaliana OX%3D3702)|
|Oglab_000003|Chr01|14610|16167|1557|121|CYP734A1|Similar to CYP734A1: Cytochrome P450 734A1 (Arabidopsis thaliana OX%3D3702)|


Une sortie plus visuelle est proposé, en utilisant en complément bedPlot.py du script bedAnalysis.py

In [3]:
%%bash
python bedPlot.py -tsv {output_bedAnalysis.tsv} -colnames REGIONS nb_gene -baseChr Chr -baseUN ChrUN -o {outname}

python: can't open file '/home/djamel/reproductibilite/test_git/integration/jupyter_directory/bedPlot.py': [Errno 2] No such file or directory


CalledProcessError: Command 'b'python bedPlot.py -tsv {output_bedAnalysis.tsv} -colnames REGIONS nb_gene -baseChr Chr -baseUN ChrUN -o {outname}\n'' returned non-zero exit status 2.

### Association des valeurs statistique de chaque k-mers aux gènes

Dans cette partie nous allons relier les valeurs statistiques des k-mers aux gènes. Comme le différentielle d'expression, ou encore la valeur de test statistique (pvalue).

#### SelectedKmerIntoGenes.py

Dans ce scirpt nous allons utilisé le fichier bed qui est créer suite à l'intersection entre le fichier d'annotation et notre fichier bam issus du mapping. Ainsi que la matrice obtenue par l'outil DEkupl, présenté dans la partie expression différentielle.

Exemple de structure du fichier diff-counts.tsv  

|tag|pvalue|meanA|meanB|log2FC|RC1|RC2|RC3|RC5|RC6|RC7|RC8|RC9|RC10|RS1|RS2|RS3|RS4|RS5|RS6|RS7|RS8|RS10|
|---|------|-----|-----|------|---|---|---|---|---|---|---|---|----|---|---|---|---|---|---|---|---|----|
|AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAC|4.20378804183321e-67|8.63539545864017|69.663581259201|3.02693059366878|6.21217947483477|4.88860953977233|8.72149434084036|9.16014388087816|8.69132810322318|12.8018115985836|13.9431607053845|5.89953533784795|7.40029614639669|74.9162700511193|72.4851902807309|71.7218111765533|68.6567983961772|60.9638382832583|72.5478816018572|63.5218693401627|68.9376846416892|73.2208875612604|
|AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGT|0.000219182949844764|0.498349895691011|5.337454146431|3.55810867156966|0|1.39674558279209|0|0|0|0|1.74289508817306|0|1.34550839025394|7.34473235795288|11.3702259263892|7.17218111765533|0|6.35039982117274|4.9464464728539|4.6479416590363|0	6.20515996281868|


Le fichier en paramètre de -diff, doit contenir au minimum une colonne tag et une colonne pvalue. Le différentiel d'expression est falcultatif, lorsqu'il est renseigné permet d'obtenir un volcano plot ainsi que la moyenne et l'écart-type pour l'ensemble des k-mers associé à un gène. Si le différentiel d'epression n'est pas renséigné dans le fichier d'entré, le script se lance et avertir l'utilisateur qu'il n'y aura pas de volcanoPot, ni de calcul du différentiel d'expression. 

In [ ]:
%%bash
python SelectedKmerIntoGene.py [-diff DIFF] [-b BED] [-s] [-o OUTPUT] [-mapq MAPQ] [-k NBKMER] [-chunk CHUNK]
                               [-baseChr BASECHR] [-baseUN BASEUN]

#### Output

Le script fourni un fichier tsv en sortie. Contenant les information liés au gènes comme la médian des pvalue de chaque k-mers, la moyenne du différentielle d'expression (log2FC), le nombre de k-mers associé, des notes du gène.

|Gene|region|START|STOP|length|nb_kmers|medianne_Pval|min_Pval|mean_Log2FC|sd_Log2FC|Similar|Note|
|----|------|-----|----|------|--------|-------------|--------|-----------|---------|-------|----|
Oglab_000002|Chr01|7074|	14927|	7853|	162|	4.6209119498895747e-07|	8.64812585511708e-13|	-0.8874876541405748|	4.163574256963421|	RABGAP22|Similar to RABGAP22: Rab GTPase-activating protein 22 (Arabidopsis thaliana OX%3D3702)|
|Oglab_000003|	Chr01|	14610|	16167|	1557|	121|	5.60450520898466e-08|	8.64812585511708e-13|	-0.9884863660805971|	4.623170079687324|	CYP734A1|Similar to CYP734A1: Cytochrome P450 734A1 (Arabidopsis thaliana OX%3D3702)|

# Approche RNA-seq classique

Cette partie a été réalisé par mon encadrante de stage, Julie Orjuela.

Nous avons utilisé TOGGle qui est une suite d'outils capables de concevoir des pipelines qui gèrent de grands ensembles de logiciels et d'utilitaires NGS.

Voici le code utilisé pour lancer TOGGle:

Voici le fichier de configuration que nous avons utilisé pour notre analyse RNA-seq classique:  config_toggle.txt

Ensuite nous avons efféctué une analyse expression différentielle. Avec l'outil DESeq2 sur nos coptages RNA-seq obtenue.

### Analyse DESeq2

Pour l'analyse de l'expression différentielle, nous avons utilisé la méthode DEkupl et une méthode DESeq2 classique.

#### Méthode DEkupl

Nous avons utilisé les mêmes outils et méthode proposée par DEkupl.

Il prend en entrée une matrice de comptage non normalisée, ainsi qu'un fichier csv avec les conditions de chaque échantillons.

Le fichier contient en première colonnes les échantillons et en seconde les conditions.

|SAMPLES|CONDITIONS|
|-------|----------|
|RC1|RC|
|RC2|RC|
|RC3|RC|
|RC5|RC|
|RC6|RC|
|RC7|RC|
|RC8|RC|
|RC9|RC|
|RC10|RC|
|RS1|RS|
|RS2|RS|
|RS3|RS|
|RS4|RS|
|RS5|RS|
|RS6|RS|
|RS7|RS|
|RS8|RS|

Ensuite l'utilisateur doit saisir les seuil de pvalue et de différentielle d'expression et en dernier paramètre le nom du fichier de sortie.

La ligne de commande:

In [5]:
%% bash
singularity exec dekupl-run.simg Rscript deseq2.r gene_count_JO_matrix.tsv sample_condition_JO.tsv 0.05 2 RC RS diff_raws_counts.tsv


bash: ligne 1: singularity : commande introuvable


CalledProcessError: Command 'b'singularity exec dekupl-run.simg Rscript deseq2.r gene_count_JO_matrix.tsv sample_condition_JO.tsv 0.05 2 RC RS diff_raws_counts.tsv\n'' returned non-zero exit status 127.

#### Output

|ID|pvalue|	meanA|meanB|log2FC|RC1	|RC10	|RC2|	RC3|	RC5|	RC6|	RC7|	RC8|	RC9|	RS1|	RS10|	RS2|	RS3|	RS4|	RS5|	RS6|	RS7|	RS8|
|--|----------|------|-------|--------|------|----|-----|------|-------|-------|-------|---------|-------|------|--------|-------|------|----|-----|------|-------|-------|
|Oglab_000896|	0.00015174035566483|	15.3992509185483|	1.34442836511834| 3.54652249966555|	15.7587397102201|	17.1151572897798|	16.5033869769884|	17.4675799610352|	17.569253647919	|11.1482208807883|	13.2423179777814|	20.3047192597427|	9.48388256267999|	12.09985528606510|	0|	0|	0|	0	|0	|0|	0|
|Oglab_000944|	0.0102792134490542|	6.68992321984196|	30.1108636235403|2.18007977253502|	4.2978381027873|	9.33554033987988|	6.287004562662240|	4.88034823553306|	0	|10.9054383346435|	8.12188770389709|16.3812516991745|	44.3661360489054|	94.7350464107319|	9.28156985420951|	19.8066977721898|	33.5739023184836|	8.16627931722220|21.2269406247195	|39.8412002654004|

#### Méthode DESeq2 classique

Pour la méthode DESeq2 classique, on utilise les mêmes paramètres, les mêmes format de fichier, c'est le traitement au niveau de l'objet DESeq2 qui change.

La ligne de commande:

In [ ]:
%% bash
singularity exec dekupl-run.simg Rscript verif_deseq2.r gene_count_JO_matrix.tsv sample_condition_JO.tsv 0.05 2 RC RS diff_raws_counts.tsv

#### Output

|ID|pvalue|	meanA|meanB|log2FC|RC1	|RC10	|RC2|	RC3|	RC5|	RC6|	RC7|	RC8|	RC9|	RS1|	RS10|	RS2|	RS3|	RS4|	RS5|	RS6|	RS7|	RS8|
|--|----------|------|-------|--------|------|----|-----|------|-------|-------|-------|---------|-------|------|--------|-------|------|----|-----|------|-------|-------|
|Oglab_000896|	0.00015174035566483|	15.3992509185483|	1.34442836511834| 3.54652249966555|	15.7587397102201|	17.1151572897798|	16.5033869769884|	17.4675799610352|	17.569253647919	|11.1482208807883|	13.2423179777814|	20.3047192597427|	9.48388256267999|	12.09985528606510|	0|	0|	0|	0	|0	|0|	0|
|Oglab_000944|	0.0102792134490542|	6.68992321984196|	30.1108636235403|2.18007977253502|	4.2978381027873|	9.33554033987988|	6.287004562662240|	4.88034823553306|	0	|10.9054383346435|	8.12188770389709|16.3812516991745|	44.3661360489054|	94.7350464107319|	9.28156985420951|	19.8066977721898|	33.5739023184836|	8.16627931722220|21.2269406247195	|39.8412002654004|

### Identification des gènes communs aux analyse

#### Intersection des fichiers tabulés contenant les gènes identifiès

On croise des fichiers au format stv, contenant au minimum une colonne contenant les varibles utilisé pour le croisement des fichiers.
Elle doit être le première colonne dans les deux fichiers croisé.
L'ordre et le nom des autres colonnes importe peu.

Mais il est important d'avoir en première colonne la variable qu'on souhaite croisé dans les deux fichiers.
Le nom de la colonne n'a pas d'impact.

Fichier 1:

|ID|pvalue|
|---|------|
|Oglab_000896|	0.00015174035566483	|
|Oglab_000944|	0.0102792134490542	|

Fichier 2:

|Gene|moyenne|
|----|-------|
|Oglab_000003|	10|
|Oglab_000004|	5	|

Voici la ligne de code nécessaire:


In [ ]:
%% bash
 python Intersect.py -wb diff_counts.tsv -wa filter_pvalue_log.tsv  

#### Output

On obtient un fichier tsv qui contient l'ensemble des informations lié au gène. 

|Gene|		pvalue|ID|	moyenne|	
|----|----------------|---------|-------|
|maker-MH_352828-exonerate_protein2genome-gene-0.0|	7.34906480821365e-06|maker-MH_352828-exonerate_protein2genome-gene-0.0	|12.14500539839015|	
